---
title: "Working with Posteriors"
output:
  rmarkdown::html_vignette:
    toc: true
    toc_depth: 3
params:
  EVAL: !r identical(Sys.getenv("NOT_CRAN"), "true")
vignette: >
  %\VignetteIndexEntry{Working with Posteriors}
  %\VignetteEngine{knitr::rmarkdown}
  %\VignetteEncoding{UTF-8}
---



## Summary statistics

We can easily customize the summary statistics reported by `$summary()` and `$print()`.

In [ ]:
%%R
fit <- cmdstanr::cmdstanr_example("schools", method = "sample")
fit$summary()

By default all variables are summaries with the follow functions:

In [ ]:
%%R
posterior::default_summary_measures()

To change the variables summarized, we use the variables argument

In [ ]:
%%R
fit$summary(variables = c("mu", "tau"))

We can additionally change which functions are used

In [ ]:
%%R
fit$summary(variables = c("mu", "tau"), mean, sd)

To summarize all variables with non-default functions, it is necessary to set explicitly set the variables argument, either to `NULL` or the full vector of variable names.

In [ ]:
%%R
fit$metadata()$model_params
fit$summary(variables = NULL, "mean", "median")

Summary functions can be specified by character string, function, or using a formula (or anything else supported by [rlang::as_function]). If these arguments are named, those names will be used in the tibble output. If the summary results are named they will take precedence.

In [ ]:
%%R
my_sd <- function(x) c(My_SD = sd(x))
fit$summary(
  c("mu", "tau"), 
  MEAN = mean, 
  "median",
  my_sd,
  ~quantile(.x, probs = c(0.1, 0.9)),
  Minimum = function(x) min(x)
)        

Arguments to all summary functions can also be specified with `.args`.

In [ ]:
%%R
fit$summary(c("mu", "tau"), quantile, .args = list(probs = c(0.025, .05, .95, .975)))

The summary functions are applied to the array of sample values, with dimension `iter_sampling`x`chains`.

In [ ]:
%%R
fit$summary(variables = NULL, dim, colMeans)

For this reason users may have unexpected results if they use `stats::var()` directly, as it will return a covariance matrix. An alternative is the `distributional::variance()` function, 
which can also be accessed via `posterior::variance()`.

In [ ]:
%%R
fit$summary(c("mu", "tau"), posterior::variance, ~var(as.vector(.x)))

Summary functions need not be numeric, but these won't work with `$print()`.

In [ ]:
%%R
strict_pos <- function(x) if (all(x > 0)) "yes" else "no"
fit$summary(variables = NULL, "Strictly Positive" = strict_pos)
# fit$print(variables = NULL, "Strictly Positive" = strict_pos)

For more information, see `posterior::summarise_draws()`, which is called by `$summary()`.


## Extracting posterior draws/samples

The [`$draws()`](https://mc-stan.org/cmdstanr/reference/fit-method-draws.html)
method can be used to extract the posterior draws in formats provided by the
[**posterior**](https://mc-stan.org/posterior/) package. Here we demonstrate
only the `draws_array` and `draws_df` formats, but the **posterior** package
supports other useful formats as well.

In [ ]:
%%R
# default is a 3-D draws_array object from the posterior package
# iterations x chains x variables
draws_arr <- fit$draws() # or format="array"
str(draws_arr)

# draws x variables data frame
draws_df <- fit$draws(format = "df")
str(draws_df)
print(draws_df)

To convert an existing draws object to a different format use the
`posterior::as_draws_*()` functions.

To manipulate the `draws` objects use the various methods described in the
posterior package [vignettes](https://mc-stan.org/posterior/articles/index.html)
and [documentation](https://mc-stan.org/posterior/reference/index.html).